# Flux Balance Analysis

## FBA

FreeFlux is capable of solving the canonical [flux balance analysis](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3108565/) (FBA) problem, which is defined as:
<div style="text-align: center">
    <img src="images/fba.gif" />
</div>
where ${\bf{S}}$ is the stoichiometric matrix, ${\bf{v}}$ is the flux vector bounded by lower bound ${{\bf{v}}_{lb}}$ and upper bounds ${{\bf{v}}_{ub}}$. The objective function could be any linear combination of fluxes, which is typically, the biomass formation.

Here we use an example of the *E. coli* metabolic network. The model file can be found [here](https://github.com/Chaowu88/freeflux/blob/main/models/ecoli/experimental_data/reactions.xlsx).

In [1]:
from freeflux import Model

MODEL_FILE = 'path/to/reactions.xlsx'

model = Model('ecoli')
model.read_from_file(MODEL_FILE)

with model.optimizer() as opt:
    opt.set_flux_bounds('all', bounds = [-100, 100]) 
    opt.set_flux_bounds('glk', bounds = [10, 10])
    opt.prepare()
    res = opt.optimize(objective = {'biom': 1})

print('objective:', res.opt_objective)

glycolysis_enzymes = ['pgi', 'pfk','fba', 'tpi', 'gapdh', 'eno', 'pk']
for enzyme in glycolysis_enzymes:
    print(enzyme, round(res.opt_fluxes[enzyme], 3))


optimizing [elapsed: 0:00:01]

objective: 1.237
pgi 9.746
pfk 8.769
fba 8.769
tpi 8.769
gapdh 16.778
eno 14.625
pk 10.808


## FVA

[Flux variable analysis](https://pubmed.ncbi.nlm.nih.gov/20920235/) (FVA) is used to find the minimum and maximum values of a flux while maintaining the objective above some fraction of its optimized value in the original FBA problem. This is formulated as:
<div style="text-align: center">
    <img src="images/fva.gif" />
</div>
where $\gamma$ $\in$ [0,1] controls how close the FBA objective should be to its optimal value $ob{j_{FBA}}$. 

FreeFlux provides a function for FVA that estimates the feasible range of fluxes which can provide a reasonable feasible region for fluxes and initial guesses during the least squares optimization. As an example, the flux ranges of the E. coli model can be estimated using the following code:

In [2]:
with model.optimizer() as opt:
    opt.set_flux_bounds('all', bounds = [-100, 100]) 
    opt.set_flux_bounds('glk', bounds = [10, 10])
    opt.prepare()
    res = opt.estimate_fluxes_range(
        objective = {'biom': 1}, 
        gamma = 0
    )

for enzyme in glycolysis_enzymes:
    print(
        enzyme, 
        [round(res.flux_ranges[enzyme][0], 3), 
         round(res.flux_ranges[enzyme][1], 3)]
    )


estimating flux ranges [elapsed: 0:00:12]

pgi [-18.946, 10.0]
pfk [0.0, 10.0]
fba [0.0, 10.0]
tpi [0.0, 10.0]
gapdh [8.055, 20.0]
eno [5.951, 20.0]
pk [0.0, 26.42]
